# Use VGG Pretrain Model to extract features for SVM

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torch.utils.data.sampler import SubsetRandomSampler
# from google.colab import drive 
from matplotlib import pyplot as plt
import math
import pandas as pd

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# Load Google Drive
# drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/Final_Project')

# Setting Seed
torch.manual_seed(1234)

cpu


## Define data loader

In [2]:
transform = transforms.Compose([
  transforms.Resize((224,224)),
  transforms.ToTensor(),
  transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225],
  ),
])

train_loader = torch.utils.data.DataLoader(datasets.ImageFolder('./Data2/train', transform=transform), batch_size=1, shuffle = False)

test_loader = torch.utils.data.DataLoader(datasets.ImageFolder('./Data2/test', transform=transform), batch_size=1, shuffle = False)

## Initialize pretrain model

In [3]:
vgg16 = models.vgg16_bn()

# freeze all layers
for param in vgg16.parameters():
    param.requires_grad = False

In [4]:
in_features = vgg16.classifier[0].in_features
vgg16.classifier = nn.Sequential(nn.Identity(in_features))
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

## Extract features

In [5]:
with torch.no_grad():
    isInit = True
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device).view(-1, 1)
        outputs = vgg16(images)[0].view(-1, 1)  # convert 1 row to 1 column
        feature_label = torch.cat((outputs, labels)).view(1, -1).numpy()  # column to row
        data_frame = pd.DataFrame(feature_label)
        data_frame.to_csv('./vgg_features_train.csv', mode='a', header=isInit, index=False)
        isInit = False
       

In [ ]:
with torch.no_grad():
    isInit = True
    for i, (images, labels) in enumerate(test_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device).view(-1, 1)
        outputs = vgg16(images)[0].view(-1, 1)  # convert 1 row to 1 column
        feature_label = torch.cat((outputs, labels)).view(1, -1).numpy()  # column to row
        data_frame = pd.DataFrame(feature_label)
        data_frame.to_csv('./vgg_features_test.csv', mode='a', header=isInit, index=False)
        isInit = False

In [ ]:
from sklearn import svm

X = pd.read_csv('./vgg_features_train.csv')
X = np.array(X)
Y = X[:,513]
X = X[:,0:512]
print(Y.shape)
print(X.shape)



In [ ]:
clf = svm.NuSVC(gamma="auto").fit(X, Y)

In [ ]:
X_test = pd.read_csv('./vgg_features_test.csv')
X_test = np.array(X_test)
Y_test = X_test[:,513]
X_test = X_test[:,0:512]
# print(Y_test.shape)
# print(X_test.shape)
outputs = clf.predict(X_test)
test = np.sum(outputs == Y_test)

print(test/len(Y_test))